# FitzHugh-Nagumo Cardiac Action Potential Analysis

This notebook explores the FitzHugh-Nagumo (FHN) model for cardiac electrophysiology, analyzing action potential generation, propagation, and refractory periods.

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.insert(0, '..')

from quantro_simulator import (
    SimulationConfig, ModelType, OverlayMode,
    create_model, RK4Integrator
)

%matplotlib inline

## 1. Action Potential Dynamics

In [ ]:
config = SimulationConfig(
    model=ModelType.FITZHUGH_NAGUMO,
    overlay_mode=OverlayMode.BASELINE,
    t_start=0.0,
    t_end=100.0,
    dt=0.01
)

model = create_model(config)
time_points, trajectory = RK4Integrator.integrate(model, lambda_param=0.3)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time series
axes[0, 0].plot(time_points, trajectory[:, 0], 'b-', linewidth=2)
axes[0, 0].set_xlabel('Time', fontsize=11)
axes[0, 0].set_ylabel('Membrane Potential (v)', fontsize=11)
axes[0, 0].set_title('Action Potential Time Series', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(time_points, trajectory[:, 1], 'r-', linewidth=2)
axes[0, 1].set_xlabel('Time', fontsize=11)
axes[0, 1].set_ylabel('Recovery Variable (w)', fontsize=11)
axes[0, 1].set_title('Recovery Dynamics', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Phase portrait
axes[1, 0].plot(trajectory[:, 0], trajectory[:, 1], 'g-', linewidth=1.5, alpha=0.7)
axes[1, 0].plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=10, label='Start')
axes[1, 0].plot(trajectory[-1, 0], trajectory[-1, 1], 'rs', markersize=10, label='End')
axes[1, 0].set_xlabel('Membrane Potential (v)', fontsize=11)
axes[1, 0].set_ylabel('Recovery (w)', fontsize=11)
axes[1, 0].set_title('Phase Portrait', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Nullclines
v_range = np.linspace(-2.5, 2.5, 200)
w_v_nullcline = v_range - v_range**3 / 3
w_w_nullcline = (v_range + 0.7) / 0.8

axes[1, 1].plot(v_range, w_v_nullcline, 'b--', linewidth=2, label='v-nullcline')
axes[1, 1].plot(v_range, w_w_nullcline, 'r--', linewidth=2, label='w-nullcline')
axes[1, 1].plot(trajectory[:, 0], trajectory[:, 1], 'k-', alpha=0.4, linewidth=1)
axes[1, 1].set_xlabel('Membrane Potential (v)', fontsize=11)
axes[1, 1].set_ylabel('Recovery (w)', fontsize=11)
axes[1, 1].set_title('Nullclines and Trajectory', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xlim(-2.5, 2.5)
axes[1, 1].set_ylim(-1, 2)

plt.tight_layout()
plt.savefig('../artifacts/fhn_action_potential.png', dpi=300)
plt.show()

## 2. Refractory Period Analysis

In [ ]:
lambda_values = np.linspace(0.0, 1.0, 20)
refractory_periods = []

for lam in lambda_values:
    config = SimulationConfig(
        model=ModelType.FITZHUGH_NAGUMO,
        overlay_mode=OverlayMode.CONTROL,
        t_end=50.0,
        dt=0.01
    )
    model = create_model(config)
    time_points, trajectory = RK4Integrator.integrate(model, lam)
    
    # Find peaks (simplified)
    v = trajectory[:, 0]
    peaks = []
    for i in range(1, len(v)-1):
        if v[i] > v[i-1] and v[i] > v[i+1] and v[i] > 0.5:
            peaks.append(time_points[i])
    
    avg_period = np.mean(np.diff(peaks)) if len(peaks) > 1 else 0
    refractory_periods.append(avg_period)

plt.figure(figsize=(10, 6))
plt.plot(lambda_values, refractory_periods, 'o-', linewidth=2, markersize=7)
plt.xlabel('Control Parameter (λ)', fontsize=12)
plt.ylabel('Refractory Period (time units)', fontsize=12)
plt.title('Refractory Period vs Control Parameter', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../artifacts/fhn_refractory_period.png', dpi=300)
plt.show()

## 3. Bifurcation Analysis

In [ ]:
lambda_vals = np.linspace(0.0, 1.0, 100)
final_states = []

for lam in lambda_vals:
    config = SimulationConfig(
        model=ModelType.FITZHUGH_NAGUMO,
        overlay_mode=OverlayMode.TIME_WARP,
        t_end=200.0,
        dt=0.01
    )
    model = create_model(config)
    _, trajectory = RK4Integrator.integrate(model, lam)
    
    # Collect last 100 points to identify attractors
    final_states.extend([(lam, v) for v in trajectory[-100:, 0]])

lambda_plot = [s[0] for s in final_states]
v_plot = [s[1] for s in final_states]

plt.figure(figsize=(12, 7))
plt.plot(lambda_plot, v_plot, 'k.', markersize=1, alpha=0.3)
plt.xlabel('Parameter λ', fontsize=12)
plt.ylabel('Membrane Potential (v)', fontsize=12)
plt.title('Bifurcation Diagram: FitzHugh-Nagumo Model', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../artifacts/fhn_bifurcation.png', dpi=300)
plt.show()

## Summary

Key findings:
- Action potential generation follows excitable dynamics
- Refractory period modulated by control parameter
- Bifurcation analysis reveals parameter-dependent behavior

**Clinical Relevance**: Understanding action potential dynamics is crucial for:
- Arrhythmia prediction
- Drug effect modeling
- Pacemaker design